# Introduction
We want to learn a function $\tau$ that maps a graph $\mathbf{G}$ and one of its node $n$ to a vector of reals:

\begin{equation}
 \tau: (\mathbf{G},n) \mapsto \mathbb{R}^m
\end{equation}

There are two broad applications of GNN. They can be classified as _graph-focused_ and _node-focused_. In graph-focused applications, $\tau$ is independent of $n$ so that $\tau: \mathbf{G} \mapsto \mathbb{R}^m$. In node-focused application, $\tau$ depends on the properties of each node $n$. GNNs are suitable for both graph- and node-focused applications.

GNNs are based on an information diffusion mechanism. A graph is processed by a set of units, each one corresponding to a node of the graph, which are linked according to the graph connectivity. The units update their states and exchange information until they reach a stable equilibrium. The output of a GNN is then computed locally at each node on the base of the unit state. The diffusion mechanism is constrained in order to ensure that a unique stable equilibrium always exists.

# I. Notations
$\mathbf{G}$ is a graph.It is a pair $(\mathbf{N},\mathbf{E})$, where $\mathbf{N}$ is the set of nodes and $\mathbf{E}$ is the set of edges. The set ${\rm ne}[n]$ stands for the neighbors of $n$ and the set ${\rm co}[n]$ stands for the edges that connect to $n$.

Nodes and edges may have labels represented by real vectors. The labels attached to node $n$ and edge $(n_1,n_2)$ will be represented by $\mathbf{l}_n$ and $\mathbf{l}_{(n_1,n_2)}$, respectively. Labels usually include features of objects related to nodes and features of the relationships between the objects.

Each graph $\mathbf{G}_i$ has a set of nodes $\mathbf{N}_i$ and a set of edges $\mathbf{E}_i$. The $j^{\rm th}$ node in the set $\mathbf{N}_i$ is referred to as $n_{ij}$ and the desired node-level target associated to $n_{ij}$ is referred to as $\mathbf{t}_{ij}$.

### Example
![Exemple of graphs](img/graphs.jpg)

$\mathbf{G}_1$ has four nodes and three edges so that 

- $\mathbf{N}_1$ is the set of nodes: $\left\{ n_{1,1} ; n_{1,2} ; n_{1,3} ; n_{1,4} \right\}$


- $\mathbf{E}_1$ is the set of edges: $\left\{ (n_{1,1},n_{1,2}) ; (n_{1,2},n_{1,3}) ; (n_{1,3},n_{1,4})\right\}$


- Each node has a target $\mathbf{t}\in\mathbb{R}^m$: $\left\{ \mathbf{t}_{1,1} ; \mathbf{t}_{1,2} ; \mathbf{t}_{1,3} ; \mathbf{t}_{1,4} \right\}$


$\mathbf{G}_2$ has five nodes and five edges so that 

- $\mathbf{N}_2 = \left\{ n_{2,1} ; n_{2,2} ; n_{2,3} ; n_{2,4} ; n_{2,5} \right\}$


- $\mathbf{E}_2 = \left\{ (n_{2,1},n_{2,2}) ; (n_{2,2},n_{2,3}) ; (n_{2,3},n_{2,4}) ; (n_{2,3},n_{2,5}) ; (n_{2,4},n_{2,5})\right\}$


- Each node has a target $\mathbf{t}\in\mathbb{R}^m$: $\left\{ \mathbf{t}_{2,1} ; \mathbf{t}_{2,2} ; \mathbf{t}_{2,3} ; \mathbf{t}_{2,4} ; \mathbf{t}_{2,5} \right\}$

The learning set $\mathcal{L}$ can be seen as all the pairs $(\mathbf{G},\mathcal{T})$ where $\mathbf{G}=(\mathbf{N}, \mathbf{E})$ is a graph and $\mathcal{T}$ is a set of pairs $\left\{ (n_i,\mathbf{t}_i) \right\}$.

# 2. The Model
The intuitive idea underlining the GNNs is that nodes in a graph represent objects or concepts, and edges represent their relationships. Each node is naturally defined by its features and the related nodes. We define the _state_ of node $n$ by $\mathbf{x}_n$. It is a multi-dimensional real vector that is based on the information contained in the neighborhood of $n$. The state $\mathbf{x}_n$ can be used to produce an _output_ $\mathbf{o}_n$.

The _local transition function_ $f_{\mathbf{w}}$, parameterized by $\mathbf{w}$, expresses the dependence of a node $n$ on its neighborhood and the _local output function_ $g_{\mathbf{w}}$ describes how the output is produced. Since the state $\mathbf{x}_n$ of node $n$ depends on its own features, as well as those of the neighborhood, it is a function of:
- The labels (features) of node $n$: $\mathbf{l}_n$.
- The labels of its connecting edges: $\mathbf{l}_{{\rm co}[n]}$.
- The state of all its neighbors: $\mathbf{x}_{{\rm ne}[n]}$.
- The labels (features) of all its neighbors: $\mathbf{l}_{{\rm ne}[n]}$.

We can therefore write that:
$$ \mathbf{x}_n =  f_{\mathbf{w}}\left( \mathbf{l}_n, \mathbf{l}_{{\rm co}[n]}, \mathbf{x}_{{\rm ne}[n]}, \mathbf{l}_{{\rm ne}[n]} \right)$$

Then, based on the state and the features of node $n$, we use $g_{\mathbf{w}}$ to calculate the ouput:
$$ \mathbf{o}_n = g_{\mathbf{w}}\left( \mathbf{x}_n, \mathbf{l}_n \right) $$

Note 1: Different notions of neighborhood can be adopted. For example, we could remove the labels $\mathbf{l}_n$ since they are already included in the state $\mathbf{x}_n$. Same for $\mathbf{l}_{{\rm ne}[n]}$, which is included in $\mathbf{x}_{\rm ne}$.

Note 2: Different transition and output functions can be used for different kind of nodes. If that's the case, an additional parameter $k$ can be used to group different the different kinds.

Let $\mathbf{x}$, $\mathbf{o}$, $\mathbf{l}$, $\mathbf{l_N}$ be the vectors constructed by stacking all the states, all the outputs, all the labels, and all the node labels, respectively. Previous equations can be rewritten in a compact form:
$$ \mathbf{x} = F_{\mathbf{w}}\left(\mathbf{x}, \mathbf{l} \right) $$

$$ \mathbf{o} = G_{\mathbf{w}}\left( \mathbf{x}, \mathbf{l_N} \right) $$

where $F_{\mathbf{w}}$ and $G_{\mathbf{w}}$ are the _global transition_ and _output functions_, respectively.

It is useful to replace the local transition function with:
$$ \mathbf{x}_n = \sum\limits_{u\in{\rm ne}[n]} h_{\mathbf{w}}\left( \mathbf{l}_n, \mathbf{l}_{(n,u)}, \mathbf{x}_u, \mathbf{l}_{u} \right), \qquad n\in\mathbf{N}$$

This transition function has been successfully used in recursive neural networks.

In order to implement the GNN model, the following items are needed:
1. A method to solve the two above equations and find the state of node $n$.
2. A learning algorithm to estimate $\mathbf{w}$ examples from the training set, in order to get $f_{\mathbf{w}}$ and $g_{\mathbf{w}}$.
3. An implementation of  $f_{\mathbf{w}}$ and $g_{\mathbf{w}}$.

## 2.1. Computation of $\mathbf{x}_n$
$\mathbf{x}_n$ can be computed using the following iterative scheme:
$$ \mathbf{x}(t+1) = F_{\mathbf{w}} \left( \mathbf{x}(t), \mathbf{l} \right) $$

where $\mathbf{x}(t)$ denotes the $t$-th iteration of $\mathbf{x}$. This dynamical system converges exponentially fast to the solution for any initial value of $\mathbf{x}(0)$. This scheme is actually the **Jacobi iterative method for solving nonlinear equations**. The outputs and the states can be computed by iterating
$$ \mathbf{x}_n(t+1) = f_{\mathbf{w}}\left( \mathbf{l}_n, \mathbf{l}_{{\rm co}[n]}, \mathbf{x}_{\rm ne}, \mathbf{l}_{{\rm ne}[n]} \right) $$
$$ \mathbf{o}_n(t) = g_{\mathbf{w}}\left( \mathbf{x}_n(t), \mathbf{l}_n \right), \quad n\in\mathcal{N} $$

The equations above can be interpreted as the representation of a network consisting of units, which compute $f_{\mathbf{w}}$ and $g_{\mathbf{w}}$. Such network is called an **encoding network**. In order to build the encoding network, each node of the graph is replaced by a unit computing the function $f_{\mathbf{w}}$. Each unit stores the current state $\mathbf{x}_n(t+1)$ using the node label and the information stored in the neighborhood. The output of node $n$ is produced by another unit, which implements $g_{\mathbf{w}}$.

![Encoding network](img/encoding_network.jpg)

When $f_{\mathbf{w}}$ and $g_{\mathbf{w}}$ are implemented by feedforward NN, the encoding network turns out to be a **recurrent neural network**. The internal connectivity is determined by the neural network architecture used to implement the unit. The external connectivity depends on the edges of the processed graph.

## 2.2. The learning algorithm
Learning in GNNs consists of estimating the parameter $\mathbf{w}$ such that $\varphi_\mathbf{w}$ approximates the data in the learning dataset. For graph-focused tasks, one special node is used for the target, whereas for node-focused tasks, in principle, the supervision can be performed on every node. The cost function can be expressed as:
$$ e_\mathbf{w} = \sum_{i=1}^p \sum_{j=1}^{q_i} (\mathbf{t}_{i,j} - \varphi_\mathbf{w}(\mathbf{G}_i, n_{i,j}))^2 $$

where :
- $p$ is the total number of graphs.
- $i$ is the $i$-th graph.
- $q_i$ is the total number of nodes of graph $i$.
- $j$ is the $j$-th node from graph $i$.
- $\mathbf{t}_{i,j}$ is the target label of node $j$ from graph $i$.

The learning algorithm is based on a gradient descent strategy and is composed of the following steps:

a) The states $\mathbf{x}_n(t)$ are iteratively updated until at time $T$ they approach the fixed point solution $\mathbf{x}(T) = \mathbf{x}$.

b) The gradient $\dfrac{\partial e_\mathbf{w}(T)}{\partial\mathbf{w}}$ is computed by **backpropagation through time**.

c) The weights $\mathbf{w}$ are updated according to the gradient computed in step b).

Backpropagation through time consists of carrying out the traditional backpropagation step on the unfolded network to compute the gradient of the cost function at time $T$ wrt all the instances of $f_\mathbf{w}$ and $g_\mathbf{w}$.

## 2.3. Transition and Output function implementations
The implementation of $g_\mathbf{w}$ does not need to fulfill any particular constraint. It is a multilayered feedforward NN (FNN). On the other hand, $f_\mathbf{w}$ plays a crucial role in the model, since its implementation determines the number and the existence of the solutions of
$$ \mathbf{x}_n =  f_{\mathbf{w}}\left( \mathbf{l}_n, \mathbf{l}_{{\rm co}[n]}, \mathbf{x}_{{\rm ne}[n]}, \mathbf{l}_{{\rm ne}[n]} \right)$$

Two possible neural networks for $f_\mathbf{w}$ are :
1. **Linear GNN**: $h_{\mathbf{w}}$ can be implemented by 
$$ h_{\mathbf{w}}\left( \mathbf{l}_n, \mathbf{l}_{(n,u)}, \mathbf{x}_u, \mathbf{l}_{u} \right)  = \mathbf{A}_{n,u}\ \mathbf{x}_u + \mathbf{b}_n$$

where $\mathbf{b}_n\in\mathbb{R}^s$ and the matrix $\mathbf{A}_{n,u}\in\mathbb{R}^{s\times s}$ are defined by the output of two FNNs, whose parameters correspond to the parameters of the GNN. More precisely, the FNNs that generates $\mathbf{A}_{n,u}$ and $\mathbf{b}_n$ are called the **transition network** and the **forcing network**, respectively.

2. **Nonlinear GNN**: in this case, $h_{\mathbf{w}}$ is realized by a multilayered FNN. However, not all parameters $\mathbf{w}$ can be used because it must ensure that $\mathbf{x}_n(t)$ converges to the stationary solution. This can be achieved by adding a penalty term $L$ to the cost function, that is:
$$ e_\mathbf{w} = \sum_{i=1}^p \sum_{j=1}^{q_i} (\mathbf{t}_{i,j} - \varphi_\mathbf{w}(\mathbf{G}_i, n_{i,j}))^2 + \beta L \left( \left\| \frac{\partial F_{\mathbf{w}}}{\partial \mathbf{x}} \right\| \right) $$